In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import countrymanagement as cm
import mccmanagement as mccm
import result

reload(fd)
reload(sd)
reload(cm)
reload(mccm)
reload(result)

print('done')

In [ ]:
import pandas as pd

#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

import dataimport as di
import pandas as pd 
   
dfTrx = di.read_file('../data/raw/'+source+'export'+date+'.csv')
dfTrx.head(5)


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#sorted_df
sorted_df = dfTrx.sort_values(by=['card_pan_id','trx_date_time'])

max=18
for i in np.arange(1,max+1,1):
    sorted_df['card_pan_id'+str(i)] = sorted_df['card_pan_id'].shift(-1*i)
    sorted_df['trx_amount'+str(i)] = sorted_df['trx_amount'].shift(-1*i)

dfTrx['previousTrxAmountSum']=0
dfTrx['nbPreviousTrx']=0
for i in np.arange(1,max+1,1):
    sorted_dfTemp1=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id'+str(i)])]
    dfTrx['nbPreviousTrx']=np.where(sorted_df.index.isin(sorted_dfTemp1.index),i,dfTrx['nbPreviousTrx'])
    dfTrx['previousTrxAmountSum']=np.where(sorted_df.index.isin(sorted_dfTemp1.index),
                                           sorted_df['trx_amount'+str(i)]+dfTrx['previousTrxAmountSum'],dfTrx['previousTrxAmountSum'])


print('nbPreviousTrx.mean()', dfTrx['nbPreviousTrx'].mean())
print(dfTrx['previousTrxAmountSum'].mean())
iv=fd.calc_iv(dfTrx,'nbPreviousTrx','Class',pr=0)
print('nbPreviousTrx iv=',iv)
dfTrx['previousTrxAmountSumBin']=pd.cut(dfTrx['previousTrxAmountSum'], bins=5)
iv=fd.calc_iv(dfTrx,'previousTrxAmountSumBin','Class',pr=0)
print('previousTrxAmountSum iv=',iv)
dfTrx['previousTrxAmountSumLog']=np.log10(1+dfTrx['previousTrxAmountSum'])
dfTrx['previousTrxAmountSumLogBin']=pd.cut(dfTrx['previousTrxAmountSumLog'], bins=5)
iv=fd.calc_iv(dfTrx,'previousTrxAmountSumLogBin','Class',pr=0)
print('previousTrxAmountSumLog iv=',iv)

dfTrx['ecom'] = np.where(dfTrx.card_entry_mode.isin([5,6,9]), 'ECOM','FTF')
dfTrx2=dfTrx[(dfTrx.ecom=='FTF')]

print('')
print('----FTF------------')
iv=fd.calc_iv(dfTrx2,'nbPreviousTrx','Class',pr=0)
print('nbPreviousTrx (FTF) iv=',iv)
dfTrx2['previousTrxAmountSumBin']=pd.cut(dfTrx2['previousTrxAmountSum'], bins=5)
iv=fd.calc_iv(dfTrx2,'previousTrxAmountSumBin','Class',pr=0)
print('previousTrxAmountSum (FTF) iv=',iv)
dfTrx2['previousTrxAmountSumLog']=np.log10(1+dfTrx2['previousTrxAmountSum'])
dfTrx2['previousTrxAmountSumLogBin']=pd.cut(dfTrx2['previousTrxAmountSumLog'], bins=5)
iv=fd.calc_iv(dfTrx2,'previousTrxAmountSumLogBin','Class',pr=0)
print('previousTrxAmountSumLog (FTF) iv=',iv)

#dfTrx[['previousTrx','previousTrxAmount']].head(5)
title='trx_amount_log10'
sns.violinplot(x='Class', y='previousTrxAmountSumLog', data=dfTrx2, palette=['#2f7ed8', '#d6604d'])
plt.title(f'{title} Distribution by Class')
plt.xlabel('Class (0 = Non-Fraud, 1 = Fraud)')
plt.ylabel(title)

plt.tight_layout()
plt.suptitle("Feature Distribution Comparison by Class", y=1.02, fontsize=16)
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def calculate_card_aggregate(max):
    print('max=',max)
    sorted_df = dfTrx.sort_values(by=['card_pan_id','trx_date_time'])

    for i in np.arange(1,max+1,1):
        sorted_df['card_pan_id'+str(i)] = sorted_df['card_pan_id'].shift(-1*i)
        sorted_df['trx_amount'+str(i)] = sorted_df['trx_amount'].shift(-1*i)

    dfTrx['previousTrxAmountSum']=0
    dfTrx['nbPreviousTrx']=0
    for i in np.arange(1,max+1,1):
        sorted_dfTemp1=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id'+str(i)])]
        dfTrx['nbPreviousTrx']=np.where(sorted_df.index.isin(sorted_dfTemp1.index),i,dfTrx['nbPreviousTrx'])
        dfTrx['previousTrxAmountSum']=np.where(sorted_df.index.isin(sorted_dfTemp1.index),
                                           sorted_df['trx_amount'+str(i)]+dfTrx['previousTrxAmountSum'],dfTrx['previousTrxAmountSum'])

    return dfTrx.copy()

def calculate_iv(dfTrx,verbose=False):
    iv=fd.calc_iv(dfTrx,'nbPreviousTrx','Class',pr=0)
    if(verbose):
        print('nbPreviousTrx iv=',iv)
    dfTrx['previousTrxAmountSumBin']=pd.cut(dfTrx['previousTrxAmountSum'], bins=5)
    iv=fd.calc_iv(dfTrx,'previousTrxAmountSumBin','Class',pr=0)
    if(verbose):
        print('previousTrxAmountSum iv=',iv)
    dfTrx['previousTrxAmountSumLog']=np.log10(1+dfTrx['previousTrxAmountSum'])
    dfTrx['previousTrxAmountSumLogBin']=pd.cut(dfTrx['previousTrxAmountSumLog'], bins=5)
    iv=fd.calc_iv(dfTrx,'previousTrxAmountSumLogBin','Class',pr=0)
    if(verbose):
        print('previousTrxAmountSumLog iv=',iv)

    dfTrx['ecom'] = np.where(dfTrx.card_entry_mode.isin([5,6,9]), 'ECOM','FTF')
    dfTrx2=dfTrx[(dfTrx.ecom=='FTF')]


    iv1=fd.calc_iv(dfTrx2,'nbPreviousTrx','Class',pr=0)
    if(verbose):
        print('nbPreviousTrx (FTF)           iv=',iv1)
    dfTrx2['previousTrxAmountSumBin']=pd.cut(dfTrx2['previousTrxAmountSum'], bins=5)
    iv=fd.calc_iv(dfTrx2,'previousTrxAmountSumBin','Class',pr=0)
    if(verbose):
        print('previousTrxAmountSum (FTF) iv=',iv)
    dfTrx2['previousTrxAmountSumLog']=np.log10(1+dfTrx2['previousTrxAmountSum'])
    dfTrx2['previousTrxAmountSumLogBin']=pd.cut(dfTrx2['previousTrxAmountSumLog'], bins=5)
    iv2=fd.calc_iv(dfTrx2,'previousTrxAmountSumLogBin','Class',pr=0)
    if(verbose):
        print('previousTrxAmountSumLog (FTF) iv=',iv2)
    print(iv1+iv2)
    return iv1+iv2

range= []
ivs=[]
for i in np.arange(5,30,1):
    df=calculate_card_aggregate(i)
    iv=calculate_iv(df,False)
    ivs.append(iv)
    range.append(i)


fd.plt_train_test(range, ivs, "iv")